### 공공 API 데이터 크롤링 
참고: https://greendreamtrre.tistory.com/268

In [ ]:
# 키 불러오기 
from google.colab import files
uploaded= files.upload()
filename= list(uploaded.keys())[0]
filename

Saving key.txt to key.txt


'key.txt'

In [ ]:
# api 키 확인 
with open(filename) as f:
  api_key = f.read()
len(api_key)

88

In [ ]:
import requests
from urllib.parse import quote

In [ ]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

# URL 파라미터 분리하기.
med_url = 'http://apis.data.go.kr/1471000/DrbEasyDrugInfoService/getDrbEasyDrugList'
My_API_Key = api_key
# get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것
queryParams = '?' + urlencode(                                  
    {
        quote_plus('ServiceKey') : My_API_Key,                    # 필수 : 서비스키 (본인의 서비스키) 
        #quote_plus('itemName') : '한미아스피린장용정100밀리그램',          
        quote_plus('efcyQesitm') : '감기'                          # 찾는 parameter   
     }
)

response = requests.get(med_url + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

In [ ]:
xmlobj

<?xml version="1.0" encoding="utf-8"?>
<response>
<header>
<resultCode>00</resultCode>
<resultMsg>NORMAL SERVICE.</resultMsg>
</header>
<body>
<numOfRows>10</numOfRows>
<pageNo>1</pageNo>
<totalCount>568</totalCount>
<items>
<item>
<entpName>(주)유한양행</entpName>
<itemName>페니라민정(클로르페니라민말레산염)</itemName>
<itemSeq>196000011</itemSeq>
<efcyQesitm>&lt;p&gt;이 약은 고초열(꽃가루 알레르기비염(코염)), 두드러기, 가려움성 피부질환(습진, 피부염, 피부가려움증, 약물발진), 알레르기 비염(코염), 혈관운동성 코염, 코감기에 의한 재채기, 콧물, 기침, 혈관운동성 부기에 사용합니다.&lt;/p&gt;</efcyQesitm>
<useMethodQesitm>&lt;p&gt;성인은 1회 1~3정(2~6 mg) 1일 2~4회 복용합니다.&lt;/p&gt;&lt;p&gt;1일 12정(24 mg)을 초과하지 마십시오.&lt;/p&gt;</useMethodQesitm>
<atpnWarnQesitm/>
<atpnQesitm>&lt;p&gt;이 약에 과민증 환자, 녹내장, 전립선비대 등 하부요로폐색(닫혀서 막힘)성 질환, 미숙아 및 신생아는 이 약을 복용하지 마십시오.&lt;/p&gt;&lt;p&gt;이 약을 복용하기 전에 3세 미만 유아, 임부 및 수유부, 고령자, 안내압(눈내부 압력)상승, 갑상샘기능항진, 협착(좁아짐)성소화성궤양 또는 유문십이지장 폐색(닫혀서 막힘), 순환계질환, 고혈압 등 심혈관계 질환, 기관지염, 기관지확장증 및 천식, 간질, 간질환, 뇌졸증, 중증(심한 증상) 관상동맥(심장동맥)부전, 발작 환자 또는 경험자는 의사 또는 약사와 상의하십시오.&lt;/p&gt;&lt;p&gt;이 약은 복용 후

In [ ]:
cold = xmlobj.findAll('itemName')
cold

[<itemName>판콜에이내복액</itemName>,
 <itemName>제로미아액</itemName>,
 <itemName>뇌선</itemName>,
 <itemName>부루펜정200밀리그램(이부프로펜)</itemName>,
 <itemName>게보린정(수출명:돌로린정)</itemName>,
 <itemName>제로바액</itemName>,
 <itemName>부루펜정400밀리그램(이부프로펜)</itemName>,
 <itemName>스코펜정400밀리그램(이부프로펜)</itemName>,
 <itemName>이부펜200밀리그람정(이부프로펜)</itemName>,
 <itemName>이텍스이부프로펜정400mg(수출명:IPRO,ILJINIBUPROPENTabs.)</itemName>]

In [ ]:
cold[0].get_text().strip()

'페니라민정(클로르페니라민말레산염)'

In [ ]:
cold_med = []
for i in range(len(cold)):
  med = cold[i].get_text().strip()
  cold_med.append(med)

In [ ]:
# 감기약
cold_med

['페니라민정(클로르페니라민말레산염)',
 '판콜에이내복액',
 '액티피드정',
 '액티피드시럽',
 '페리나정',
 '갈타제산(베타-갈락토시다제(아스퍼길루스))',
 '부루펜정200밀리그램(이부프로펜)',
 '액티페린정',
 '마스질정(클레마스틴푸마르산염)',
 '부루펜정400밀리그램(이부프로펜)']

### 여러개의 증상으로 api 호출

In [ ]:
sym  = ['두통', '염증', '상처', '과식', '구토', '복통','발열']

In [ ]:
# 여러개 증상의 api 호출하는 함수 
def call_sym(sym):
  med_total = []
  med_url = 'http://apis.data.go.kr/1471000/DrbEasyDrugInfoService/getDrbEasyDrugList'
  My_API_Key = api_key
  # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것
  for i in range(len(sym)):
    queryParams = '?' + urlencode(                                  
        {
            quote_plus('ServiceKey') : My_API_Key,                    # 필수 : 서비스키 (본인의 서비스키) 
            #quote_plus('itemName') : '한미아스피린장용정100밀리그램',          
            quote_plus('efcyQesitm') : sym[i]                          # 찾는 parameter   
        }
    )
    response = requests.get(med_url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
    med_list = xmlobj.find_all('itemName')

    name_list = []
    for n in med_list:
      name = n.text
      name_list.append(name)
    
    med_total.append(name_list)
  return med_total

In [ ]:
dictionary = dict(zip(sym, med_total))
dictionary

{'과식': ['활명수',
  '보화소합원(대환,소환)',
  '베스타제정',
  '베스타제당의정',
  '건위황정',
  '잘젠정',
  '제스판골드정',
  '베로타정',
  '다이제틴정',
  '리포유정'],
 '구토': ['활명수',
  '아네모정',
  '겔포스현탁액(인산알루미늄겔)',
  '게루삼정',
  '보화소합원(대환,소환)',
  '보나링에이정(디멘히드리네이트)',
  '건위황정',
  '제스판골드정',
  '포리부틴정(트리메부틴말레산염)',
  '스파부틴정100밀리그램(트리메부틴말레산염)'],
 '두통': ['판콜에이내복액',
  '뇌선',
  '낙센정(나프록센)',
  '부루펜정200밀리그램(이부프로펜)',
  '게보린정(수출명:돌로린정)',
  '바로론정(클로닉신리시네이트)',
  '아나프록스정(나프록센나트륨)',
  '부루펜정400밀리그램(이부프로펜)',
  '스코펜정400밀리그램(이부프로펜)',
  '크로낙정(클로닉신리시네이트)'],
 '발열': ['판콜에이내복액',
  '제로미아액',
  '뇌선',
  '부루펜정200밀리그램(이부프로펜)',
  '게보린정(수출명:돌로린정)',
  '제로바액',
  '부루펜정400밀리그램(이부프로펜)',
  '스코펜정400밀리그램(이부프로펜)',
  '이부펜200밀리그람정(이부프로펜)',
  '이텍스이부프로펜정400mg(수출명:IPRO,ILJINIBUPROPENTabs.)'],
 '복통': ['포리부틴정(트리메부틴말레산염)',
  '스파부틴정100밀리그램(트리메부틴말레산염)',
  '타라부틴정(트리메부틴말레산염)(수출명:뉴트리소젠정)',
  '베부틴정(트리메부틴말레산염)(수출명:영일트리메부틴정,TRISPASMTab)',
  '티메틴정(트리메부틴말레산염)(수출명:OZIMETINETab.)',
  '신일트리메부틴말레산염정',
  '토레스정(트리메부틴말레산염)',
  '레큐틴정(트리메부틴말레산염)',
  '벤즈날정(트리메부틴말레산염)',
  '타라부틴정200밀리그램(트리메부틴말레산염)'],
 '상처': [

In [ ]:
dictionary['과식']

['활명수',
 '보화소합원(대환,소환)',
 '베스타제정',
 '베스타제당의정',
 '건위황정',
 '잘젠정',
 '제스판골드정',
 '베로타정',
 '다이제틴정',
 '리포유정']

In [ ]:
# 3개만 호출 
def call_sym1(sym):
  med_total = []
  med_url = 'http://apis.data.go.kr/1471000/DrbEasyDrugInfoService/getDrbEasyDrugList'
  My_API_Key = api_key
  # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것
  for i in range(len(sym)):
    queryParams = '?' + urlencode(                                  
        {
            quote_plus('ServiceKey') : My_API_Key,                    # 필수 : 서비스키 (본인의 서비스키) 
            #quote_plus('itemName') : '한미아스피린장용정100밀리그램',          
            quote_plus('efcyQesitm') : sym[i]                          # 찾는 parameter   
        }
    )
    response = requests.get(med_url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
    med_list = xmlobj.find_all('itemName')

    name_list = []
    for n in med_list[:3]:
      name = n.text
      name_list.append(name)
    
    med_total.append(name_list)
  return med_total

In [ ]:
call_sym1(sym)

[['판콜에이내복액', '뇌선', '낙센정(나프록센)'],
 ['쎄레스톤지크림', '키모랄에스정', '기모타부정(수출명:케이티정)'],
 ['에어신신파스', '성광멸균바세린거즈(백색바셀린)', '미보(MEBO)연고'],
 ['활명수', '보화소합원(대환,소환)', '베스타제정'],
 ['활명수', '아네모정', '겔포스현탁액(인산알루미늄겔)'],
 ['포리부틴정(트리메부틴말레산염)',
  '스파부틴정100밀리그램(트리메부틴말레산염)',
  '타라부틴정(트리메부틴말레산염)(수출명:뉴트리소젠정)'],
 ['판콜에이내복액', '제로미아액', '뇌선']]